# **Installing all of the packages**

In [1]:
import os
import torch
import pandas as pd
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, TaskType,prepare_model_for_kbit_training
from datasets import load_dataset
from sql_metadata import Parser
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from tqdm import tqdm
from swanlab.integration.huggingface import SwanLabCallback

/home/vipuser/anaconda3/envs/DTS_SQL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2797/522962034.py:10: DeprecationWarning: The module 'huggingface' is deprecated and will be removed in future versions. Please update your imports to use 'transformers' instead.
  from swanlab.integration.huggingface import SwanLabCallback


# **Connect to wandb**

In [2]:
# os.environ["WANDB_PROJECT_40G_newPrompt"]="deepseek_full_finetuning"

# **Loading the model**

In [3]:
model_name = "/home/LLM_para/deepspeek_code"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant = True
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
#     quantization_config=bnb_config,
    attn_implementation="flash_attention_2",
    torch_dtype = torch.bfloat16,
    device_map='auto',
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████| 2/2 [01:09<00:00, 34.63s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model = prepare_model_for_kbit_training(model) 
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaLinearScalingRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

# **Loading the dataset**

In [7]:
# 拆解训练集
# 提供每次训练token数
import copy 
import re
max_token = 1800
#  保证特殊字符能被识别
accept_toekn = max_token - 150
#获取REF表数据
tab_str = ""
schema_slicing = []
df = pd.read_csv("./mydataset_new/table_schema_Reference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['Reference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"

    
print(len(schema_slicing))

# print(ReF_group[5])
# print("#############################")
# print(ReF_group[6])


#获取noREF表数据
df = pd.read_csv("./mydataset_new/table_schema_noReference_cropped.csv",encoding="utf-8")
for index, row in tqdm(df.iterrows(), total=len(df)):
    table_group = row['noReference_group']
    tab_str_elder = copy.deepcopy(tab_str)
    tab_str += table_group + "\n"
    check_token = tokenizer(f"{tab_str}", add_special_tokens=False)
    if len(check_token["input_ids"]) > accept_toekn:
#         print("+++++++++++")
        schema_slicing.append(tab_str_elder)
        tab_str = table_group + "\n"
        
        
if len(tab_str) > 0:
    schema_slicing.append(tab_str)

print(len(schema_slicing))
# print(schema_slicing[6])
# print("#############################")
# print(schema_slicing[7])
# print(schema_slicing[8])
create_table_pattern = re.compile(r'CREATE TABLE `[^`]+` \([^;]+\);')
table_name_group = []
test_total = set()
count = 0
for piece in schema_slicing:
    create_table_statements = create_table_pattern.findall(piece)
    temp_list = []
    for table in create_table_statements:
        table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
        temp_list.append(table_name.lower())
        test_total.add(table_name.lower())
        count+=1
    print(temp_list)
    table_name_group.append(temp_list)
# print(count)
print(len(test_total))
#     table_name_group.append()

100%|████████████████████████████████████████████████████████████████████████████████| 142/142 [00:00<00:00, 480.42it/s]


6


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 528.93it/s]

8
['manufacturers', 'products', 'student', 'plays_games', 'sportsinfo', 'actor', 'entrepreneur', 'people', 'ref_hotel_star_ratings', 'ref_attraction_types', 'hotels', 'tourist_attractions', 'street_markets', 'shops', 'museums', 'royal_family', 'theme_parks', 'visits', 'photos', 'staff', 'tourist_attraction_features', 'wrestler', 'elimination']
['business', 'category', 'checkin', 'neighbourhood', 'review', 'tip', 'ref_detention_type', 'ref_incident_type', 'addresses', 'students', 'teachers', 'assessment_notes', 'behavior_incident', 'detention', 'student_addresses', 'students_in_detention', 'film', 'film_market_estimation', 'catalogs']
['catalog_structure', 'catalog_contents', 'catalog_contents_additional_attributes', 'routes', 'airports', 'stadium', 'game', 'injury_accident', 'physician', 'department', 'affiliated_with', 'trained_in', 'patient', 'nurse', 'appointment', 'prescribes', 'block', 'room', 'on_call', 'stay']
['undergoes', 'buildings', 'office_locations', 'region', 'party', 'me

In [11]:
def fill_dataset_template(schema_pad,question_pad,answer_pad):
     message = {
                "user": f" I want you to act as a relation extraction robot for a sample SQL table. You need to return the tables related to the user's input question and tables.##Question:{question_pad}Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema_pad}",
                "assistant": f"```Reference Table\n-- Tables: {answer_pad};\n```<|EOT|>",
            }
     return message
#  #None#

In [12]:
df = pd.read_csv("./mydataset_new/full_finetuning_dataset_cropped.csv",encoding="utf-8")
outer_index = 0
message_group = []
for i in range(len(df['db_id'])):
    question = df['question'][i]
    query = df['query'][i]
    correct_table = df['correct_tables'][i]
    correct_table_list = correct_table.split(", ")
    dict_target_table_slice = {}
    for target_table in correct_table_list:
        for index, table_name_slicing in enumerate(table_name_group): 
            if target_table.lower() in table_name_slicing:
                dict_target_table_slice[target_table] = index
                break    
    sorted_relation = sorted(dict_target_table_slice.items(), key=lambda x: x[1])
    for index, piece in enumerate(schema_slicing):
        this_selected = []
        for key, value in sorted_relation:
            if index == value:
                this_selected.append(key)
#          不给与选择提示
        if len(this_selected) == 0:
            message = fill_dataset_template(piece,question,"#None#")
        else:
            this_selected_str = ', '.join(this_selected)
            message = fill_dataset_template(piece,question,this_selected_str)          
        message_group.append([message["user"],message["assistant"]])
       
new_df = pd.DataFrame(message_group, columns = ['user','assistant'])
new_df.to_csv('full_finetuning_dataset_Dmyidea.csv', index=False ,encoding="UTF-8")


print(len(message_group))

13888


In [13]:
# data_files = {"train": "./schema/full_finetuning_dataset.csv", "validation": "./schema/validation_dataset_formatted.csv"}
data_files = {"train": "full_finetuning_dataset_Dmyidea.csv"}
dataset = load_dataset('csv', data_files=data_files)

Generating train split: 13888 examples [00:00, 21197.92 examples/s]


**Filtering rows with max tokens**

In [14]:
data_test = pd.read_csv("full_finetuning_dataset_Dmyidea.csv")
max_len = 0
count = 0
def print_Prompt_len(training_dataset):
    max_len = 0
    for index, row in tqdm(training_dataset.iterrows(), total=len(df)):
        _user = row['user']
        _assistant = row['assistant']
        messages = [
        {"role": "user", "content": _user},
        {"role": "assistant", "content": _assistant},
        ]
        if index == 1:
            print(messages)
        instruction = tokenizer(_user,add_special_tokens=False)
        response = tokenizer(_assistant, add_special_tokens=False) 
        input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
        if len(input_ids) > max_len:
            max_len = len(input_ids)
    return max_len
print (print_Prompt_len(data_test))

  3%|██▌                                                                             | 56/1736 [00:00<00:06, 272.45it/s]

[{'role': 'user', 'content': " I want you to act as a relation extraction robot for a sample SQL table. You need to return the tables related to the user's input question and tables.##Question:Find the total revenue of companies of each founder.Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:CREATE TABLE `business` (\r\n  bid INT PRIMARY KEY,\r\n  business_id TEXT,\r\n  name TEXT,\r\n  full_address TEXT,\r\n  city TEXT,\r\n  latitude TEXT,\r\n  longitude TEXT,\r\n  review_count INT,\r\n  is_open INT,\r\n  rating REAL,\r\n  state TEXT\r\n);\nCREATE TABLE `category` (\r\n  id INT PRIMARY KEY,\r\n  business_id TEXT REFERENCES business(business_id),\r\n  category_name TEXT\r\n);\nCREATE TABLE `checkin` (\r\n  cid INT PRIMARY KEY,\r\n  business_id TEXT REFERENCES business(business_id),\r\n  count INT,\r\n  day TEXT\r\n);\nCREATE TABLE `neighbourhood` (\r\n  id INT PRIMARY KEY,\r\n  business_

13888it [00:50, 274.48it/s]                                                                                             

1766


In [15]:
def formatting_prompts_func(training_dataset):
    output_texts = []
    for i in range(len(training_dataset['user'])):
        _user = training_dataset['user'][i]
        _assistant = training_dataset['assistant'][i]
        messages = [
        {"role": "user", "content": _user},
        {"role": "assistant", "content": _assistant},
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False)
        output_texts.append(text)
    return output_texts

In [16]:
response_template = "### Response:" #deepseek
# response_template = "[/INST]" #mistral"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# **Training Config**

In [19]:
lora_r = 64
lora_alpha = 32
lora_dropout = 0.1
output_dir = "./SFT_schema_Myidea"
num_train_epochs = 3
bf16 = True
overwrite_output_dir = True
per_device_train_batch_size = 2
per_device_eval_batch_size = 20
gradient_accumulation_steps = 16
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 5e-5
weight_decay = 0.01
lr_scheduler_type = "cosine"
warmup_ratio = 0.01
max_grad_norm = 0.3
group_by_length = True
auto_find_batch_size = False
save_steps = 50
logging_steps = 50
load_best_model_at_end= False
packing = False
save_total_limit=3
neftune_noise_alpha=5
report_to="wandb"
max_seq_length = 2100 #set based on the maximum number of tokens

In [20]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "v_proj",
        "k_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head"
    ],
    task_type=TaskType.CAUSAL_LM,
)

In [21]:
swanlab_callback = SwanLabCallback(
    project="ENG_SFT_T2QTabOnly0",
    experiment_name="Deepspeek-9B-2epo",
    description="使用Deepspeek模型微调spider改数据集。",
    config={
        "model": "Deepspeek",
        "dataset": "full_finetuning_dataset.csv",
    },
)

In [22]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
#     evaluation_strategy=evaluation_strategy,
    max_grad_norm = max_grad_norm,
    auto_find_batch_size = auto_find_batch_size,
    save_total_limit = save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="none",
    neftune_noise_alpha= neftune_noise_alpha
)

In [23]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
#     eval_dataset=dataset['validation'],
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=max_seq_length,
    packing=packing,
    callbacks=[swanlab_callback],
)

Map: 100%|███████████████████████████████████████████████████████████████| 13888/13888 [00:11<00:00, 1249.07 examples/s]
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [24]:
trainer.train()

swanlab: Tracking run with swanlab version 0.3.20                                  
swanlab: Run data will be saved locally in /home/code/chat_SQL/main_verify/Deepspeek_c_V/swanlog/run-20240913_163123-a3b1799d
swanlab: 👋 Hi winhong, welcome to swanlab!
swanlab: Syncing run Deepspeek-9B-2epo_Sep13_16-31-23 to the cloud
swanlab: 🌟 Run `swanlab watch /home/code/chat_SQL/main_verify/Deepspeek_c_V/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@winhong/ENG_SFT_T2QTabOnly0
swanlab: 🚀 View run at https://swanlab.cn/@winhong/ENG_SFT_T2QTabOnly0/runs/128b0h2txe1f320mtdkl9


Step,Training Loss
50,1.931300
100,0.039600
150,0.026200
200,0.022200
250,0.016500
300,0.011600
350,0.014000
400,0.012700
450,0.012400
500,0.008900


/home/vipuser/anaconda3/envs/DTS_SQL/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/vipuser/anaconda3/envs/DTS_SQL/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/vipuser/anaconda3/envs/DTS_SQL/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/vipuser/anaconda3/envs/DTS_SQL/lib/python3.10/site-packages/peft/ut

TrainOutput(global_step=1302, training_loss=0.08384265333916398, metrics={'train_runtime': 19362.9517, 'train_samples_per_second': 2.152, 'train_steps_per_second': 0.067, 'total_flos': 2.7996112380351283e+18, 'train_loss': 0.08384265333916398, 'epoch': 3.0})

In [25]:
output_dir = os.path.join("./", "final_checkpoint_SFT_deepspeek_Myidea")
trainer.model.save_pretrained(output_dir)

/home/vipuser/anaconda3/envs/DTS_SQL/lib/python3.10/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
